# Business Analytics - Unit 04

## Lab 02 - tidy data

### Introduction

The purpose of this lab is to introduce how Python can be used to tidy messy datasets. After completing this lab the student should be able to use Python to:

1. Column headers are values, not variable names
2. Variables are stored in both rows and columns
3. Multiple types of observational units are stored in the same table
4. A single observational unit is stored in multiple tables

Before we start tidying data, let's import the pandas library

In [2]:
import pandas as pd

### 1. Column headers are values, not variable names

First we need to some messy data. For demonstration purposes we'll create a sample messy dataset and then work on tidying it up. The sample dataset will mimic the examples from Hadley Wickham's [article](http://vita.had.co.nz/papers/tidy-data.pdf) on tidy data.

The sample data is being created by calling pandas' DataFrame function. To build the dataframe we pass a Python dictionary to create it. The dictionary datatype is indicated by the use of the curly braces '{}'. The format for dictionary is **key*:*value**; in our example the key is the column name and the value is the list of data elements for the column. We pass the values as a list, as indicated by the square brackets '[]'.

In [3]:
messy = pd.DataFrame({'religion':['Agnostic', 'Atheist', 'Buddhist', 'Catholic'],
                      '$0-10k':[27, 12, 27, 418],
                      '$10k-20k':[34, 27, 21, 617],
                      '$20k-30k':[60, 37, 30, 732],
                      '$30k-40k':[81, 52, 34, 670],
                      '$40k-50k':[76, 35, 33, 638],
                      '$50k-75k':[137, 70, 58, 1116]})
messy.head()

,$0-10k,$10k-20k,$20k-30k,$30k-40k,$40k-50k,$50k-75k,religion
0,27,34,60,81,76,137,Agnostic
1,12,27,37,52,35,70,Atheist
2,27,21,30,34,33,58,Buddhist
3,418,617,732,670,638,1116,Catholic


As indidated in Hadley's article and in the [slides](https://docs.google.com/presentation/d/1JeA_eBGSHWJn6Udnv7Hh8xcwQRaJVfuqcVehtShoLo4/edit?usp=sharing) for the unit this sample dataset is designed for presentation, where variables form both the rows and columns, and column headers are values, not variable names. To tidy this data we will need turn the columns into rows.

In [4]:
tidy = pd.melt(messy, id_vars=['religion'], value_vars=['$0-10k', '$10k-20k', '$20k-30k', '$30k-40k', '$40k-50k', '$50k-75k'],\
        var_name = 'income', value_name = 'frequency').sort_values(by = ['religion', 'income'])
tidy.head()

,religion,income,frequency
0,Agnostic,$0-10k,27
4,Agnostic,$10k-20k,34
8,Agnostic,$20k-30k,60
12,Agnostic,$30k-40k,81
16,Agnostic,$40k-50k,76


To create tidy data from our original messy dataset, we call the built-in *melt* function from the pandas library. 

According the pandas documentation, the melt function:
*Massages a DataFrame into a format where one or more columns are identifier variables (id_vars), while all other columns, considered measured variables (value_vars), are “unpivoted” to the row axis, leaving just two non-identifier columns, ‘variable’ and ‘value’.*

Essentially we tell Python the 'religion' column is the identifier, the variable is the income level, and finally the count of income by religion is the value.

Finally, we sort the dataset so everything is grouped appropriately. 

One last thing, which isn't required, is we write the tidy dataset to a csv file; for future use.

**Side Note**: the numbers you see in the left most column are the row indexes for the dataframe,

In [21]:
tidy.to_csv('output/tidy_1.csv')

### 2. Each different observation of that variable should be in a different row

As with the previous example our first step will be to create text data. We'll also include a new library this time, numpy. Numpy is a lbrary used for managing array structures. For our purpuoses we'll use it to generate a null value in fields that should be left blank.

In [5]:
import numpy as np
messy = pd.DataFrame({'id':['MX17004','MX17004','MX17004','MX17004','MX17004','MX17004'],
                      'year':[2010, 2010, 2010, 2010, 2010, 2010],
                      'month':[1,1,2,2,3,3],
                      'element':['tmax', 'tmin','tmax', 'tmin','tmax', 'tmin'],
                      'd1':[np.nan, np.nan, np.nan, np.nan, np.nan, np.nan],
                      'd2':[np.nan, np.nan, 27.3, 14.4, np.nan, np.nan],
                      'd3':[np.nan, np.nan, 24.1, 14.4, np.nan, np.nan],
                      'd4':[np.nan, np.nan, np.nan, np.nan, np.nan, np.nan],
                      'd5':[np.nan, np.nan, np.nan, np.nan, 32.1, 14.2],
                      'd6':[np.nan, np.nan, np.nan, np.nan, np.nan, np.nan]})
messy.head()

,d1,d2,d3,d4,d5,d6,element,id,month,year
0,NaN,NaN,NaN,NaN,NaN,NaN,tmax,MX17004,1,2010
1,NaN,NaN,NaN,NaN,NaN,NaN,tmin,MX17004,1,2010
2,NaN,27.3,24.1,NaN,NaN,NaN,tmax,MX17004,2,2010
3,NaN,14.4,14.4,NaN,NaN,NaN,tmin,MX17004,2,2010
4,NaN,NaN,NaN,NaN,32.1,NaN,tmax,MX17004,3,2010


As indicated in the slides for this unit, we will need to shape this data so that it is organized around the id for the weather station and a single date. 

The first step to cleaning up this data is to melt it with id_vars of id, year, month, and element. The value_vars will be the day values. Because there are so many blank values, we are removing them with the dropna function call. The argument in the function call means the dataset will be changed in place with the call and you do not have to assign the results to a new dataset.

In [6]:
molten = pd.melt(messy, id_vars=['id', 'year', 'month', 'element'], value_vars=[ 'd1', 'd2', 'd3', 'd4', 'd5', 'd6'], var_name = 'day')
molten.dropna(inplace = True)
molten.head()

,id,year,month,element,day,value
8,MX17004,2010,2,tmax,d2,27.3
9,MX17004,2010,2,tmin,d2,14.4
14,MX17004,2010,2,tmax,d3,24.1
15,MX17004,2010,2,tmin,d3,14.4
28,MX17004,2010,3,tmax,d5,32.1


To format the date we are going to need a helper function that will take the values in the year, month, and day columns and create a date in the yyyy-mm-dd format. 

In [8]:
def create_date(row):    
    return '{:d}-{:02d}-{:02d}'.format(row['year'], row['month'], int(row['day'][1:]))

The function we defined above accepts a row of data from a pandas dataframe as an input. We pull the year, month, and day information from the row and apply the format function to it and return a string in the format of yyyy-mm-dd. 

In [9]:
molten['date'] = molten.apply(create_date,axis=1)
molten = molten[['id', 'element','value','date']]
molten.head()

,id,element,value,date
8,MX17004,tmax,27.3,2010-02-02
9,MX17004,tmin,14.4,2010-02-02
14,MX17004,tmax,24.1,2010-02-03
15,MX17004,tmin,14.4,2010-02-03
28,MX17004,tmax,32.1,2010-03-05


The apply function allows us to call another function across an axis in a dataframe. In our example appove we call the apply function and pass our helper function and assign the results to the 'date' column in the pandas dataframe. When axis is equal to 1, we are telling Python that the function we are passing it applies to each row in the dataframe.

In [12]:
tidy = molten.groupby('id').apply(pd.DataFrame.pivot,
                                  index='date',
                                  columns='element',
                                  values='value')
tidy.head()

element             tmax  tmin
id      date                  
MX17004 2010-02-02  27.3  14.4
        2010-02-03  24.1  14.4
        2010-03-05  32.1  14.2

To keep the id column, we need to use the groupby() function to group by the id and then apply pivot() inside each group.

In [13]:
tidy.reset_index(inplace=True)
tidy.head()

element,id,date,tmax,tmin
0,MX17004,2010-02-02,27.3,14.4
1,MX17004,2010-02-03,24.1,14.4
2,MX17004,2010-03-05,32.1,14.2


Finally, we'll reset the index on the dataframe to set the id for each row in the dataframe - since the previous example only displays the id for each group. In the last code block we'll write the results to a csv file incase we need to process the results at a later date.

In [9]:
tidy.to_csv('output/tidy_2.csv')